In [2]:
from devtools import debug
from dotenv import load_dotenv

%load_ext autoreload
%autoreload 2

load_dotenv(verbose=True)
!export PYTHONPATH=":./python"

from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools import PythonREPLTool
from langchain import hub
from langchain.agents import AgentExecutor

from python.ai_core.llm import get_llm

In [3]:
instructions = """You are an agent designed to write and execute python code to answer questions.
You have access to a python REPL, which you can use to execute python code.
If you get an error, debug your code and try again.
Only use the output of your code to answer the question. 
You might know the answer without running any code, but you should still run the code to get the answer.
If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.
"""
from langchain.agents import AgentExecutor, create_tool_calling_agent

llm = get_llm(llm_id="gpt_35_openai") # works
#llm = get_llm(llm_id="gpt_35_edenai")

base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)
tools = [PythonREPLTool()]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

2024-07-18 17:09:23.974 | INFO     | python.ai_core.llm:get_llm:418 - get LLM:'gpt_35_openai' -configurable: False - streaming: False


In [4]:
agent_executor.invoke({"input": "Using the tools you have, calculate the 10th fibonacci number"})



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.



Invoking: `Python_REPL` with `def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)

fibonacci(10)`


The 10th Fibonacci number is 55.

> Finished chain.


{'input': 'Using the tools you have, calculate the 10th fibonacci number',
 'output': 'The 10th Fibonacci number is 55.'}

In [6]:
query = """
Create a plot with time in X and random values between 10 et  10 in Y.
Generate Streamlit code. 
"""
agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...

Invoking: `Python_REPL` with `import streamlit as st
import numpy as np
import matplotlib.pyplot as plt

x = np.linspace(0, 10, 100)
y = np.random.uniform(-10, 10, 100)

fig, ax = plt.subplots()
ax.plot(x, y)

st.pyplot(fig)`


I have generated the Streamlit code to create a plot with time on the X-axis and random values between -10 and 10 on the Y-axis. 

```python
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt

x = np.linspace(0, 10, 100)
y = np.random.uniform(-10, 10, 100)

fig, ax = plt.subplots()
ax.plot(x, y)

st.pyplot(fig)
```

You can run this code in a Streamlit app to display the plot.

> Finished chain.


{'input': '\nCreate a plot with time in X and random values between 10 et  10 in Y.\nGenerate Streamlit code. \n',
 'output': 'I have generated the Streamlit code to create a plot with time on the X-axis and random values between -10 and 10 on the Y-axis. \n\n```python\nimport streamlit as st\nimport numpy as np\nimport matplotlib.pyplot as plt\n\nx = np.linspace(0, 10, 100)\ny = np.random.uniform(-10, 10, 100)\n\nfig, ax = plt.subplots()\nax.plot(x, y)\n\nst.pyplot(fig)\n```\n\nYou can run this code in a Streamlit app to display the plot.'}